# Trope media scanner v 1.0

<figure style="text-align:center;">
  <img src="https://i.kym-cdn.com/entries/icons/original/000/005/710/logo.jpg" width="100" alt="Image Alt Text">
  <figcaption style="text-align:center;">You'll spend so much time browsing that site that you can say, <a href="https://tvtropes.org/pmwiki/pmwiki.php/JustForFun/TVTropesWillRuinYourLife">"TV Tropes ruined my life"</a></figcaption>
</figure>

# Contents

# Table of Contents
0. [Introduction](#introduction)
   1. [What is TVTropes?](#a-what-is-tvtropes)
   2. [Problem Statement](#b-problem-statement)
   3. [Goal](#c-goal)
1. [Data Importation](#data-importation)
    1. [Imports (libraries)](#1-imports-libraries)
    2. [Scraping Episodes](#2-scraping-episodes)
    3. [Scraping Tropes](#3-scraping-tropes)
2. [Explanatory Data Analysis](#Explanatory-Data-Analysis)
    1. [EDA Set-up](#EDA-Set-up)

# 0 - Introduction 

## A. What is TVTropes?

TV Tropes is a wiki website that collects and describes common plot devices, character archetypes, and other storytelling elements. It is a valuable resource for writers, filmmakers, and other storytellers, as well as fans of popular culture who want to learn more about the stories they love. Some examples of tropes that are covered on TV Tropes include [the Hero's Journey](https://tvtropes.org/pmwiki/pmwiki.php/Main/TheHerosJourney), [Chekhov's Gun](https://tvtropes.org/pmwiki/pmwiki.php/Main/ChekhovsGun), and [Coming of Age story](https://tvtropes.org/pmwiki/pmwiki.php/Main/ComingOfAgeStory).

## B. Problem Statement

Identifying tropes in creative works can be a time-consuming and challenging task, especially for long or complex works. This can make it difficult for writers, filmmakers, and other storytellers to identify and use tropes effectively, and to avoid clichés. Additionally, fans of popular culture may have difficulty learning more about the tropes used in their favorite stories.

A trope scanner can help to solve this problem by automating the process of identifying tropes in creative works. This can save writers, filmmakers, and other storytellers a significant amount of time, and can help them to create more engaging and original stories. Additionally, a trope scanner can make it easier for fans of popular culture to learn more about the tropes used in their favorite stories.

Here are some specific examples of how a trope scanner can be used:
- A writer can use a trope scanner to identify and use tropes effectively in their work. For example, a writer could use a trope scanner to find examples of the Hero's Journey trope in popular movies, and then use those examples to inform their own writing.
- A filmmaker can use a trope scanner to identify and avoid clichés in their films. For example, a filmmaker could use a trope scanner to find examples of the [Deus ex Machina](https://tvtropes.org/pmwiki/pmwiki.php/Main/DeusExMachina) trope, and then avoid using that trope in their own films.
- A fan of popular culture can use a trope scanner to learn more about the tropes used in their favorite stories. For example, a fan of superhero movies could use a trope scanner to find examples of the [Secret Identity](https://tvtropes.org/pmwiki/pmwiki.php/Main/SecretIdentity) trope, and then learn more about the history and significance of that trope.

## C. Goal

The ultimate aim of this project is to create a bot that can populate a TVTropes wiki article with a serviceable first draft for a new piece of media. To do this, this bot needs to do the following:

A. Ingest some media <br>
B. Create a summary of the media <br>
C. Identify main characters <br>
D. Identify tropes present in the media <br>
E. Create sub sections for the tropes <br>

However, because of time restrictions, this project will only aim to create a multiclass classification model that can identify whether certain tropes are present in a piece of media. The target is an accuracy of above 0.9.

The tropes to be used as classes are:
* [insert tropes as needed]
* [insert tropes as needed]
* [insert tropes as needed]


A stretch goal for the project is to initialise an LLM model to generate text for steps B and D in the previously mentioned workflow. 

# 1 - Imports (libraries)

In [1]:
# Import basic libraries
import pandas as pd
import numpy as np

In [2]:
# Import scraping libraries
import requests
from bs4 import BeautifulSoup

In [3]:
# Import regex
import re
import string

In [4]:
# Import visualisation libraries
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

In [39]:
#browser-based scripts
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [5]:
# some display adjustments to account for the fact that we have many columns
# and some columns contain many characters

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 800)

# 2 - Scraping Episodes

In [6]:
# base wiki url

wiki_url = 'https://simpsons.fandom.com/wiki/'

In [7]:
#checking to see if site is up

response = requests.get(wiki_url)
print(response)

<Response [200]>


In [8]:
# list of seasons to scrape

seasons_list = ['Season_1'] # ivan, i've currently only put season 1 but feel free to try adding more if you want

In [9]:
# list of episode urls to scrape

episodes_urls = []

for season in seasons_list:
    url = wiki_url + season
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    episode_table = soup.find('table', class_='wikitable')
    for i, row in enumerate(episode_table.find_all('tr')):
        if i != 0:
            for i, links in enumerate(row.find_all('a')):
                if i == 1:
                    episodes_urls.append(links.get('href'))


In [10]:
episodes_urls

['/wiki/Simpsons_Roasting_on_an_Open_Fire',
 '/wiki/Bart_the_Genius',
 '/wiki/Homer%27s_Odyssey',
 '/wiki/There%27s_No_Disgrace_Like_Home',
 '/wiki/Bart_the_General',
 '/wiki/Moaning_Lisa',
 '/wiki/The_Call_of_the_Simpsons',
 '/wiki/The_Telltale_Head',
 '/wiki/Life_on_the_Fast_Lane',
 '/wiki/Homer%27s_Night_Out',
 '/wiki/The_Crepes_of_Wrath',
 '/wiki/Krusty_Gets_Busted',
 '/wiki/Some_Enchanted_Evening']

According to the wiki there are indeed only 13 episodes in Season 1, so in short it works. Let's just remove the /wiki/ from the front since we have it at the end of our base url.

In [11]:
episodes_urls = [x[6:] for x in episodes_urls]
episodes_urls

['Simpsons_Roasting_on_an_Open_Fire',
 'Bart_the_Genius',
 'Homer%27s_Odyssey',
 'There%27s_No_Disgrace_Like_Home',
 'Bart_the_General',
 'Moaning_Lisa',
 'The_Call_of_the_Simpsons',
 'The_Telltale_Head',
 'Life_on_the_Fast_Lane',
 'Homer%27s_Night_Out',
 'The_Crepes_of_Wrath',
 'Krusty_Gets_Busted',
 'Some_Enchanted_Evening']

In [12]:
def scrape_clean(section):
    text = section.get_text()
    text = text.replace('\n','')
    text = text.replace('\t','')
    text = text.replace('[]','')
    text = text.replace('\u2028','')
    text = text.replace('\x93','')
    text = text.replace('\x94','')
    text = text.replace('\x85','')
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

In [13]:
# now to scrape text from each episode page

episode_list = []
full_story = []
wiki_text = {}

for episode in episodes_urls:
    url = wiki_url + episode
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all <h2> elements on the page
    section_headings = soup.find_all("h2")

    # Get the contents
    for i, heading in enumerate(section_headings):
        text = scrape_clean(heading)

        if i == 0:
            episode_title = text
            episode_list.append(episode_title)
            print(episode_title)
            
        
        if text == 'Full Story' or text == 'Story':  
            # i noticed that one episode was missing and it's because 
            # that page called this section story instead of full story
            # hence the second clause
            
            paragraphs = heading.find_all_next("p")
            full_text = ''
            
            for paragraph in paragraphs:
                # Check if a new header is detected and break the loop
                
                if paragraph.find("h2"):
                    break
                
                paragraph_text = scrape_clean(paragraph)
                
                if len(paragraph_text)>0:
                    full_text += ''.join(paragraph_text)
                    
            print(f'Length of full text is {len(full_text)}.')
            full_story.append(full_text)

for key, value in zip(episode_list, full_story):
    wiki_text.update({key:value})
    


Simpsons Roasting on an Open Fire
Length of full text is 16425.
Bart the Genius
Length of full text is 8577.
Homers Odyssey
Length of full text is 8285.
Theres No Disgrace Like Home
Length of full text is 11110.
Bart the General
Length of full text is 6838.
Moaning Lisa
Length of full text is 6520.
The Call of the Simpsons
Length of full text is 6506.
The Telltale Head
Length of full text is 4875.
Life on the Fast Lane
Length of full text is 5991.
Homers Night Out
Length of full text is 3088.
The Crepes of Wrath
Length of full text is 2872.
Krusty Gets Busted
Length of full text is 10301.
Some Enchanted Evening
Length of full text is 10211.


Going to call one episode just to check how it looks:

In [14]:
wiki_text['Simpsons Roasting on an Open Fire']

'Homer hastily drives the Family Sedan with Marge and Maggie through a snowcovered street They are late for Bart and Lisas Christmas pageant at Springfield Elementary School After crashing into a pile of snow at the front of Springfield Elementary School park Homer and Marge stumble through a full audience to find their seats Homer and Marge sit down just in time to see Principal Skinner introduce the second grade The children do a presentation of different Santa Clauses from around the world such as Ralph Wiggum as Hotei Oshō Dewey Largo introduces Lisa playing Tawanga the Santa Claus of the South Seas Lisa’s dance causes awe throughout the crowd Lisa performs a skilled dance with lit torches performing tricks such as throwing it around herself – Lisa also wears seethrough clothing effectively ‘revealing Lisa completely at the end of her act Skinner eventually introduces the fourth grade singing Christmas carols Jingle Bells is sung first and after the first verse Bart picks up his vo

Seems to work fine, so the scraping of the season was successful.

In [15]:
# Convert the dictionary to a DataFrame
wiki_df = pd.DataFrame(list(wiki_text.items()), columns=['Episode Title', 'Full Story'])

# Export the DataFrame to a CSV file
wiki_df.to_csv('simpsons_episode_data2.csv', index=False)

# Display the DataFrame
wiki_df

Episode Title  \
0   Simpsons Roasting on an Open Fire   
1                     Bart the Genius   
2                      Homers Odyssey   
3        Theres No Disgrace Like Home   
4                    Bart the General   
5                        Moaning Lisa   
6            The Call of the Simpsons   
7                   The Telltale Head   
8               Life on the Fast Lane   
9                    Homers Night Out   
10                The Crepes of Wrath   
11                 Krusty Gets Busted   
12             Some Enchanted Evening   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

Next step: scraping trope pages.

# 3 - Scraping Tropes

This section is a bit more complicated because basically we need to get the full descriptive text of each trope that is present in each episode. Actually that might get pretty intense so maybe instead of doing EVERY trope, we should do the top 10.

The method for scraping: 
1. Access the main TV show recap page
2. Find the season(s) for scraping
3. Extract the URLs for the seasons to a list
4. Iterate through the list to find the top 10 tropes and extract those to a list too 
5. Finally, iterate through the trope list and save the text for analysis

In [16]:
# base tvtropes url
tvt_url = 'https://tvtropes.org'

In [17]:
# base tv show url

recap_url = 'https://tvtropes.org/pmwiki/pmwiki.php/Recap/TheSimpsons'

In [18]:
#checking to see if site is up

response = requests.get(wiki_url)
print(response)

<Response [200]>


In [19]:
# We can use the same seasons list as previously

In [20]:
driver = webdriver.Chrome()

# Navigate to the webpage
driver.get(recap_url)

# Wait for the page to load
time.sleep(2)

# scroll once to get past the ad
bottom_element = driver.find_element(By.CSS_SELECTOR,'div.folderlabel.toggle-all-folders-button')
bottom_element.location_once_scrolled_into_view

# use selenium to click on the 'open/close all folders' button
driver.find_element(By.CSS_SELECTOR,'div.folderlabel.toggle-all-folders-button').click()

# Use Beautiful Soup to parse the page using the 'lxml' library
soup = BeautifulSoup(driver.page_source, 'lxml')

# Close the browser session (we don't need it any more)
driver.quit()

seasons = soup.find_all('div', class_ = 'folder')

In [21]:
# empty list of urls to scrape
episodes_urls = []

# iterate through seasons

for season in seasons:
    for episode in season.find_all('a', class_ = 'twikilink'):
        if 'TheSimpsonsS' in str(episode):
            episode_url = episode.get('href')
            print(episode_url)
            episodes_urls.append(episode_url)


/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E1SimpsonsRoastingOnAnOpenFire
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E2BartTheGenius
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E3HomersOdyssey
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E4TheresNoDisgraceLikeHome
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E5BartTheGeneral
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E6MoaningLisa
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E7TheCallOfTheSimpsons
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E8TheTelltaleHead
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E9LifeOnTheFastLane
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E10HomersNightOut
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E11TheCrepesOfWrath
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E12KrustyGetsBusted
/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E13SomeEnchantedEvening
/pmwiki/pmwiki.php/Recap/TheSimpsonsS2E1BartGetsAnF
/pmwiki/pmwiki.php/Recap/TheSimpsonsS2E2SimpsonAndDelilah
/pmwiki/pmwiki.php/Recap/TheSimpsonsS2E3TreehouseOfHorror
/pmwiki/pmwiki.php/Recap/TheSimpsonsS2E4TwoCarsInEveryGarageAndThreeEyesOn

While we can choose mutiple episodes, we can just stick to the first episode as an example

In [22]:
episodes_urls[0]

'/pmwiki/pmwiki.php/Recap/TheSimpsonsS1E1SimpsonsRoastingOnAnOpenFire'

On top of finding the URLs for scraping tropes, we also need to see which tropes are present in each episode. This is important because it's how we're going to be testing if the trope is present or not. 

In [23]:
# now to find the links for tropes

trope_urls = []
trope_list = []
trope_recap = []

epi_trope = {}
season_1 = [x for x in episodes_urls if 'S1E' in x]

for episode in season_1: 
    tropes = []
    url = tvt_url + episode
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    trope_twikilinks = soup.find_all('a', class_ = 'twikilink')

    for trope in trope_twikilinks:
        if 'TheSimpsons' not in str(trope):
            trope_url = trope.get('href')
            trope_text = trope.get_text()
            trope_urls.append(trope_url)
            tropes.append(trope_text)


    # Find all 'li' elements
    list_items = soup.find_all('li')

    for item in list_items:
        # Find the 'a' tag with class 'twikilink'
        desired_tag = item.find('a', class_='twikilink')

        if desired_tag:

            # Find the text next to the link
            recap_text = item.get_text(strip=True)
            recap_text = recap_text.lstrip(':').strip()
            recap_text = re.findall(r'\: (.*)', recap_text)

            trope_recap.append(recap_text)
    
    trope_list.append(tropes)

for key, value in zip(episode_list, trope_list):
    epi_trope.update({key:value})
            

In [24]:
epi_trope['Simpsons Roasting on an Open Fire']

["The '80s",
 'pony',
 'Mall Santa',
 '13 Is Unlucky',
 'All Girls Like Ponies',
 'Wise Beyond Their Years',
 'Always Someone Better',
 'Running Gag',
 'Bad People Abuse Animals',
 'Body Wipe',
 'Bread, Eggs, Milk, Squick',
 'Characterisation Click Moment',
 'Wise Beyond Their Years',
 'Children Are Innocent',
 'Christmas Episode',
 "Daddy's Girl",
 'Dasher, Dancer, Prancer and Nixon',
 'Trope Namer',
 'Nixon',
 'Department of Redundancy Department',
 'Description Cut',
 "Didn't Think This Through",
 'Dragged by the Collar',
 'Early-Bird Cameo',
 'voice',
 'Early Installment Character-Design Difference',
 'Early-Installment Weirdness',
 'somewhat different',
 'The Tracey Ullman Show',
 'Depending on the Writer',
 '"everything-is-religious" extremist',
 'Flanderization',
 'Earn Your Happy Ending',
 'Establishing Character Moment',
 "Daddy's Girl",
 'Eternal Recurrence',
 'Everyone Has Standards',
 'Expy Coexistence',
 'The Smurfs',
 'Captain Ersatz',
 'Fantasy Pantheon',
 'First-Episode

In [25]:
# Convert the dictionary to a DataFrame
episode_tropes_df = pd.DataFrame(list(epi_trope.items()), columns=['Episode Title', 'Tropes'])

episode_tropes_df.head(1)

,Episode Title,Tropes
0,Simpsons Roasting on an Open Fire,"[The '80s, pony, Mall Santa, 13 Is Unlucky, All Girls Like Ponies, Wise Beyond Their Years, Always Someone Better, Running Gag, Bad People Abuse Animals, Body Wipe, Bread, Eggs, Milk, Squick, Characterisation Click Moment, Wise Beyond Their Years, Children Are Innocent, Christmas Episode, Daddy's Girl, Dasher, Dancer, Prancer and Nixon, Trope Namer, Nixon, Department of Redundancy Department, Description Cut, Didn't Think This Through, Dragged by the Collar, Early-Bird Cameo, voice, Early Installment Character-Design Difference, Early-Installment Weirdness, somewhat different, The Tracey Ullman Show, Depending on the Writer, ""everything-is-religious"" extremist, Flanderization, Earn Your Happy Ending, Establishing Character Moment, Daddy's Girl, Eternal Recurrence, Everyone Has Standards, Expy Coexistence, The Smurfs, Captain Ersatz, Fantasy Pantheon, First-Episode Twist, Foreshadowing, Gift of the Magi Plot, Hammerspace Hair, Internal Reveal, Irony, It's All Junk, Single Tear, I ""Uh"" You, Too, Jerkass, Kick the Dog, Licked by the Dog, Mall Santa, Mistaken for Cheating, Obnoxious In-Laws, Oh, Crap!, Mall Santa, One-Book Author, Popcultural Osmosis Failure, Rousing Speech, Tiny Tim, Rapid-Fire Interrupting, Rousing Speech, Shout-Out, Strapped to an Operating Table, Goldfinger, Tiny Tim, Charlie Brown, The Smurfs (1981), ""Who's Tiny Tim?"", a closeup of the loudspeaker and the phrase Attention, all personnel , this isn't the first show, The Waltons, Sickeningly Sweet, Song Parody, Batman, Surprisingly Realistic Outcome, name, Christmas Miracle, Tempting Fate, Titled After the Song, Touché, Unwitting Instigator of Doom, Vapor Wear, Wide-Eyed Idealist, Wise Beyond Their Years, Wrong Genre Savvy, very first episode, gets a happy ending after all, Yank the Dog's Chain]"


In [26]:
combined_simpsons_df = wiki_df.merge(episode_tropes_df, how = 'outer', on = 'Episode Title')

In [27]:
combined_simpsons_df.head(1)

Episode Title  \
0  Simpsons Roasting on an Open Fire   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [28]:
# Export the DataFrame to a CSV file
combined_simpsons_df.to_csv('simpsons_episode_data2.csv', index=False)


In [29]:
# little bit of cleaning to remove duplicates and links to other media

trope_urls = set(trope_urls)
trope_urls = [x for x in trope_urls if 'Main' in x]

In [30]:
len(trope_urls)

554

In [31]:
# now to scrape text from each trope page

trope_list = []
full_desc = []
trope_text = {}

for trope in trope_urls:
    url = tvt_url + trope
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    heading = soup.find('h1')
    trope_name = scrape_clean(heading)
    trope_list.append(trope_name.strip())
    
    sections = soup.find_all('div', class_ = 'article-content retro-folders')
    
    for section in sections:
        paragraphs = section.find_all_next('p')
        full_text = ''

    for paragraph in paragraphs:
        # Check if a new header is detected and break the loop

        if paragraph.find("h2"):
            break

        paragraph_text = scrape_clean(paragraph)

        if len(paragraph_text)>0:
            full_text += ''.join(paragraph_text)

    print(f'Length of full text is {len(full_text)}.')
    full_desc.append(full_text)

for key, value in zip(trope_list, full_desc):
    trope_text.update({key:value})
    

Length of full text is 3002.
Length of full text is 1048.
Length of full text is 967.
Length of full text is 10242.
Length of full text is 2788.
Length of full text is 1803.
Length of full text is 1032.
Length of full text is 2756.
Length of full text is 2064.
Length of full text is 8360.
Length of full text is 2982.
Length of full text is 1925.
Length of full text is 4408.
Length of full text is 2888.
Length of full text is 3401.
Length of full text is 2355.
Length of full text is 917.
Length of full text is 744.
Length of full text is 3008.
Length of full text is 1493.
Length of full text is 4062.
Length of full text is 5989.
Length of full text is 2369.
Length of full text is 3714.
Length of full text is 1587.
Length of full text is 954.
Length of full text is 2430.
Length of full text is 2733.
Length of full text is 4971.
Length of full text is 2905.
Length of full text is 2472.
Length of full text is 1258.
Length of full text is 3014.
Length of full text is 3011.
Length of full te

Length of full text is 6319.
Length of full text is 6512.
Length of full text is 2720.
Length of full text is 1582.
Length of full text is 1385.
Length of full text is 3238.
Length of full text is 57398.
Length of full text is 3579.
Length of full text is 7007.
Length of full text is 1716.
Length of full text is 3253.
Length of full text is 1737.
Length of full text is 3342.
Length of full text is 2836.
Length of full text is 1488.
Length of full text is 1719.
Length of full text is 671.
Length of full text is 1706.
Length of full text is 955.
Length of full text is 2011.
Length of full text is 1529.
Length of full text is 4088.
Length of full text is 1781.
Length of full text is 1277.
Length of full text is 1526.
Length of full text is 1031.
Length of full text is 7447.
Length of full text is 2309.
Length of full text is 1332.
Length of full text is 3323.
Length of full text is 1249.
Length of full text is 3854.
Length of full text is 3487.
Length of full text is 1190.
Length of full 

Now out of the ~500 tropes, we can actually pick one to see whether it has successfully scraped.

In [32]:
import random
test_trope = random.sample(trope_list, 1)[0]
print(test_trope)
trope_text[test_trope]

Abandoned Catchphrase


'Character Catchphrases are a great and simple way to help a characters image But sometimes catchphrases dont stick Perhaps the writers were experimenting with one early on but eventually decided against it Or perhaps a catchphrase was done to death and is no longer funny Or sometimes a character simply evolves and the catchphrase no longer fits the character'

Since it works, we can now input the trope and its description into a dataframe.

In [33]:
# Convert the dictionary to a DataFrame
tropes_df = pd.DataFrame(list(trope_text.items()), columns=['Trope Name', 'Trope Description'])

tropes_df.head()

Trope Name  \
0                                                         Smooch of Victory   
1                                                    The Many Deaths of You   
2                                                       Appeal to Tradition   
3  The Reason You Suck Speech                    aka Reason You Suck Speech   
4                                                      Magical Security Cam   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [34]:
tropes_df['Text Length'] = tropes_df['Trope Description'].map(len)

In [35]:
tropes_df.head()

Trope Name  \
0                                                         Smooch of Victory   
1                                                    The Many Deaths of You   
2                                                       Appeal to Tradition   
3  The Reason You Suck Speech                    aka Reason You Suck Speech   
4                                                      Magical Security Cam   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [36]:
tropes_df = tropes_df.loc[tropes_df['Text Length']!= 0]

In [37]:
tropes_df['Text Length'].describe()

count       552.000000
mean       3322.565217
std        8132.603616
min         286.000000
25%        1339.250000
50%        2141.500000
75%        3345.500000
max      133288.000000
Name: Text Length, dtype: float64

In [38]:
# Export the DataFrame to a CSV file
tropes_df.to_csv('tropes.csv', index=False)

[Link to the first EDA Notebook](./EDA_Notebook_1.ipynb)